In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

from datetime import datetime
import zipfile
import shutil
import os
import glob
import json
from tqdm.notebook import tqdm
#import cv2
import random

In [2]:
#!pip install scikit-learn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, ElasticNetCV
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#import statsmodels.api as sm
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, roc_curve
from sklearn.metrics import precision_score, recall_score

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC, SVR                 # 비선형을 처리하기 위한 알고리즘(커널 방식) -> 고차원(평면->입체)으로 mapping시킴
from sklearn.svm import LinearSVC, LinearSVR     # 선형을 처리하기 위한 알고리즘
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

#### 원본데이터

In [4]:
HQ_coating = pd.read_csv("C:\\Users\\owner\\PythonWork\\Final_project\\TalkFile_HQ_coating2.csv")

#### 이상치 제거 데이터

In [17]:
HQ_coating = pd.read_csv("C:\\Users\\owner\\PythonWork\\Final_project\\coating_part.csv")

In [18]:
HQ_coating_ML = HQ_coating[['part','damage', 'supercategory_name', 'HQ']]
HQ_coating_ML.head()

,part,damage,supercategory_name,HQ
0,Bumper,Crushed,VAN,1.97
1,Door,Separated,VAN,4.15
2,Bumper,Scratched,VAN,1.97
3,Fender,Crushed,VAN,2.80
4,Fender,Separated,VAN,2.80


## 이상치 제거

In [19]:
HQ_coating_ML.sort_values("HQ", ascending=False)

,part,damage,supercategory_name,HQ
151654,Roof,Scratched,SUV,5.95
151657,Roof,Crushed,Full-size car,5.95
136375,Roof,Scratched,SUV,5.95
168398,Roof,Scratched,SUV,5.95
168397,Roof,Breakage,SUV,5.95
...,...,...,...,...
151580,Side mirror,Separated,Compact car,0.15
78255,Side mirror,Crushed,Mid-size car,0.15
78151,Side mirror,Crushed,Mid-size car,0.15
358203,Side mirror,Breakage,City car,0.14


In [20]:
HQ_coating_ML[HQ_coating_ML["HQ"] == 100]

,part,damage,supercategory_name,HQ


In [8]:
HQ_coating_ML = HQ_coating_ML[HQ_coating_ML["HQ"] != 100]

In [9]:
HQ_coating_ML = HQ_coating_ML[HQ_coating_ML["HQ"] != 0]

In [10]:
HQ_coating_ML

,part,damage,supercategory_name,HQ
0,Bumper,Crushed,VAN,1.97
1,Door,Separated,VAN,4.15
2,Bumper,Scratched,VAN,1.97
3,Fender,Crushed,VAN,2.80
4,Fender,Separated,VAN,2.80
...,...,...,...,...
504484,Door,Separated,City car,2.19
504485,Bumper,Separated,Compact car,1.24
504486,Bumper,Crushed,Compact car,1.24
504487,Bumper,Scratched,Compact car,1.24


## X, y로 나누기

In [21]:
X = HQ_coating_ML[["part","damage","supercategory_name"]]
X

,part,damage,supercategory_name
0,Bumper,Crushed,VAN
1,Door,Separated,VAN
2,Bumper,Scratched,VAN
3,Fender,Crushed,VAN
4,Fender,Separated,VAN
...,...,...,...
497399,Door,Separated,City car
497400,Bumper,Separated,Compact car
497401,Bumper,Crushed,Compact car
497402,Bumper,Scratched,Compact car


In [22]:
y = HQ_coating_ML[["HQ"]]
y

,HQ
0,1.97
1,4.15
2,1.97
3,2.80
4,2.80
...,...
497399,2.19
497400,1.24
497401,1.24
497402,1.24


## X 데이터 레이블 인코딩

In [23]:
label_encoder = LabelEncoder()

# 데이터프레임의 모든 열을 범주형으로 변환
X = X.astype(str)

# 각 열에 대해 LabelEncoder 적용
for column in X.columns:
    X[column] = label_encoder.fit_transform(X[column])


In [24]:
X

,part,damage,supercategory_name
0,1,1,5
1,2,3,5
2,1,2,5
3,3,1,5
4,3,3,5
...,...,...,...
497399,2,3,0
497400,1,3,1
497401,1,1,1
497402,1,2,1


## X 데이터 원핫인코딩 

In [25]:
X_one = pd.get_dummies(data=X,columns=["part","damage","supercategory_name"])

In [26]:
X_one

,part_0,part_1,part_2,part_3,part_4,part_5,part_6,part_7,part_8,part_9,damage_0,damage_1,damage_2,damage_3,supercategory_name_0,supercategory_name_1,supercategory_name_2,supercategory_name_3,supercategory_name_4,supercategory_name_5
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497399,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
497400,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
497401,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
497402,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


## train, test 나누기(레이블 인코딩)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, test_size=0.3)

## train, test 나누기(원핫인코딩)

In [28]:
X_one_train, X_one_test, y_one_train, y_one_test = train_test_split(X_one, y, random_state=42, test_size=0.3)

## train, test 잘 섞였나 확인

In [29]:
X_train.groupby("part").count()

,damage,supercategory_name
part,,
0,10176,10176
1,150604,150604
2,79210,79210
3,95414,95414
4,6,6
5,1631,1631
6,5913,5913
7,424,424
8,1161,1161


In [30]:
X_test.groupby("part").count()

,damage,supercategory_name
part,,
0,4300,4300
1,64802,64802
2,34114,34114
3,40574,40574
4,1,1
5,686,686
6,2601,2601
7,180,180
8,519,519


## Decesion Tree(레이블인코딩)

In [35]:
dt_model = DecisionTreeRegressor(random_state=155, max_depth=50)

result = dt_model.fit(X_train,y_train)

print("훈련점수",result.score(X_train, y_train))
print("테스트점수",result.score(X_test, y_test))

# 훈련점수 0.4530639256481461
# 테스트점수 0.45104494119274496

훈련점수 0.4740892748056146
테스트점수 0.47716091322230014


## Decesion Tree(원핫인코딩)

In [34]:
dt_one_model = DecisionTreeRegressor(random_state=155, max_depth=100)

result = dt_one_model.fit(X_one_train,y_train)

print("훈련점수",result.score(X_one_train, y_train))
print("테스트점수",result.score(X_one_test, y_test))

# 훈련점수 0.4526752719273631
# 테스트점수 0.4506320425412357

훈련점수 0.4740892748056147
테스트점수 0.47716091322230014


## Random Forest 레이블 인코딩

In [36]:
rnf_model = RandomForestRegressor(random_state=42)

result = rnf_model.fit(X_train,y_train)

print("훈련점수",result.score(X_train, y_train))
print("테스트점수",result.score(X_test, y_test))

# 훈련점수 0.31058619138600485
# 테스트점수 0.28726244602492823

C:\Users\owner\AppData\Local\Temp\ipykernel_8576\3310767846.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rnf_model.fit(X_train,y_train)


훈련점수 0.47408717150606416
테스트점수 0.4771507097876396


## Random Forest 원핫인코딩

In [120]:
rnf_one_model = RandomForestRegressor(random_state=42)

result = rnf_one_model.fit(X_one_train,y_train)

print("훈련점수",result.score(X_one_train, y_train))
print("테스트점수",result.score(X_one_test, y_test))

# 훈련점수 0.4530813125266584
# 테스트점수 0.4510332003205494

C:\Users\owner\AppData\Local\Temp\ipykernel_6668\4072472921.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rnf_one_model.fit(X_one_train,y_train)


훈련점수 0.4530813125266584
테스트점수 0.4510332003205494


## Gradient Boosting 레이블인코딩

In [121]:
GB_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=10, random_state=0)

result = GB_model.fit(X_train, y_train)

print("훈련점수",result.score(X_train, y_train))
print("테스트점수",result.score(X_test, y_test))

# 훈련점수 0.39225357055343
# 테스트점수 0.3907456732897886

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련점수 0.39225357055343
테스트점수 0.3907456732897886


## Gradient Boosting 원핫인코딩

In [140]:
GB_one_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=10, random_state=0)

result = GB_one_model.fit(X_one_train, y_train)

print("훈련점수",result.score(X_one_train, y_train))
print("테스트점수",result.score(X_one_test, y_test))

# 훈련점수 0.3904549443418073
# 테스트점수 0.3890485767510494

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련점수 0.3904549443418073
테스트점수 0.3890485767510494


## XGBoost 레이블 인코딩

In [124]:
import xgboost as xgb

In [137]:
XGB_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=100, random_state=0, loss='ls')

result = XGB_model.fit(X_train, y_train)

print("훈련점수:", result.score(X_train, y_train))
print("테스트점수:", result.score(X_test, y_test))

# 훈련점수: 0.39225357055343
# 테스트점수: 0.3907456732897886

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(


훈련점수: 0.3923806397716384
테스트점수: 0.39081074346164235


## XGBoost  원핫인코딩

In [139]:
XGB_one_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=100, random_state=0, loss='ls')

result = XGB_one_model.fit(X_one_train, y_train)

print("훈련점수:", result.score(X_one_train, y_train))
print("테스트점수:", result.score(X_one_test, y_test))

# 훈련점수: 0.3923806397716385
# 테스트점수: 0.39080639301320985

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(


훈련점수: 0.3923806397716385
테스트점수: 0.39080639301320985


## GridSearchCV 레이블 인코딩

In [141]:
from sklearn.model_selection import GridSearchCV

In [150]:
depth_list = range(1,20)

train_acc = []
test_acc = []

param_grid = {
    'learning_rate': [0.01],
    'max_depth': 10,
    'random_state': [0]}

model = GradientBoostingRegressor()

for depth in depth_list:
    Grid_model = GridSearchCV(model, param_grid)
    Grid_model.fit(X_train, y_train)
    
    train_acc.append(Grid_model.score(X_train,y_test))
    test_acc.append(Grid_model.score(X_test,y_test))
    
result = {"depth":depth_list,
         "Train_acc":train_acc,
         "Test_acc":test_acc}

result = pd.DataFrame(result)
result

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array w

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array w

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array w

KeyboardInterrupt: 

In [153]:
model = GradientBoostingRegressor()

param_grid = {
    'learning_rate': [0.01],
    'max_depth': [10],
    'random_state': [0]}

Grid_model = GridSearchCV(model, param_grid)
result = Grid_model.fit(X_train, y_train)

print("훈련점수:", result.score(X_train, y_train))
print("테스트점수:", result.score(X_test, y_test))

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array w

훈련점수: 0.39225357055343
테스트점수: 0.3907456732897886


## GridSearchCV 원핫 인코딩

In [154]:
model = GradientBoostingRegressor()

param_grid = {
    'learning_rate': [0.01],
    'max_depth': [10],
    'random_state': [0]}

Grid_model = GridSearchCV(model, param_grid)
result = Grid_model.fit(X_one_train, y_train)

print("훈련점수:", result.score(X_one_train, y_train))
print("테스트점수:", result.score(X_one_test, y_test))

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array w

훈련점수: 0.3904549443418073
테스트점수: 0.3890485767510494


## SVM  레이블인코딩

In [19]:
from sklearn.svm import SVR

In [ ]:
# 최적의 모델 찾기

# SVR 모델 객체 생성
sv_regressor = SVR()

# 탐색할 파라미터 그리드 정의
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1.0, 10.0],
    'epsilon': [0.01, 0.1, 1.0]
}

# GridSearchCV 객체 생성
grid_search = GridSearchCV(sv_regressor, param_grid, cv=5)

# 데이터를 사용하여 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 최적의 파라미터 출력
print("Best Parameters:", grid_search.best_params_)


C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
SVR_model = SVR(kernel='linear', C=10, epsilon=0.01)

result = SVR_model.fit(X_train, y_train)

print("훈련점수:", result.score(X_one_train, y_train))
print("테스트점수:", result.score(X_one_test, y_test))

C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
